In [3]:
import json
import sqlite3

import chromadb
import numpy as np
from chromadb.utils import embedding_functions
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential

In [4]:
TRAIN_CUTOFF = 2711  # 2424 2155 1880 1572 1016 817 502 260

In [6]:
# db = "../data/arxiv_papers.db"
# last_interested = find_last_interested_entry(db)
# print(json.dumps(last_interested, indent=4))

In [7]:
def get_data(database_path):
    # Create a database connection
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()
    cursor.row_factory = sqlite3.Row

    # SQL query to find the last occurrence of "interested" = 1
    query = """
    SELECT paper_id, concise_summary, interested FROM papers 
    ORDER BY paper_id ASC 
    LIMIT 2711;
    """

    try:
        cursor.execute(query)
        articles = cursor.fetchall()
        if articles:
            print(f"Got {len(articles)}.")
            return articles
        else:
            print("No interested entries found.")
            return nil
    except sqlite3.Error as e:
        print("Database error:", e)
    finally:
        conn.close()

In [8]:
def get_embedding(paper_id, vdb_path="../data/arxiv_embeddings.chroma"):
    vdb = chromadb.PersistentClient(vdb_path)
    sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
        model_name="all-MiniLM-L6-v2"
    )
    embedding_func = sentence_transformer_ef
    vectors = vdb.get_or_create_collection(
        name="arxiver", embedding_function=embedding_func
    )

    res = vectors.get(ids=[paper_id], limit=1, include=["embeddings"])
    # print(res)
    # print(f'{res["ids"][0]} {res["embeddings"][0]}')
    return res["embeddings"][0]

In [9]:
articles = get_data(db)
X_article = []
y_article = []
for article in articles:
    # print(
    #     f'{article["paper_id"]}, {article["interested"]}\n{article["concise_summary"]}'
    # )
    X_article.append(get_embedding(article["paper_id"]))
    y_article.append(article["interested"])

# print(X_article[:3])
# print(y_article[:3])

X = np.array(X_article)
y = np.array(y_article)

# print(X[:1])
# print(y[:1])
print(X.shape, y.shape)

Got 2711.


Add of existing embedding ID: http://arxiv.org/abs/1706.03762v7


(2711, 384) (2711,)


In [10]:
print(len(X[0]))

384


In [11]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [12]:
def build_model(hp):
    model = Sequential()
    model.add(
        Dense(
            units=hp.Int("units1", min_value=32, max_value=512, step=32),
            activation="relu",
            input_shape=(X_train.shape[1],),
        )
    )
    model.add(
        Dropout(rate=hp.Float("dropout1", min_value=0.0, max_value=0.5, step=0.1))
    )
    model.add(
        Dense(
            units=hp.Int("units2", min_value=32, max_value=256, step=32),
            activation="relu",
        )
    )
    model.add(
        Dropout(rate=hp.Float("dropout2", min_value=0.0, max_value=0.5, step=0.1))
    )
    model.add(Dense(1, activation="sigmoid"))

    # Compile model
    hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [13]:
tuner = RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=20,
    executions_per_trial=2,
    directory="my_dir",
    project_name="arxiv_article_embeddings",
)

In [14]:
from tensorflow.keras.callbacks import EarlyStopping

stop_early = EarlyStopping(monitor="val_loss", patience=5)

tuner.search(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 20 Complete [00h 00m 17s]
val_accuracy: 0.9447004795074463

Best val_accuracy So Far: 0.950460821390152
Total elapsed time: 00h 09m 17s


In [15]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(X_train, y_train, epochs=50, validation_split=0.2)

Epoch 1/50
55/55 [==============================] - 4s 19ms/step - loss: 0.2584 - accuracy: 0.9446 - val_loss: 0.2051 - val_accuracy: 0.9447
Epoch 2/50
55/55 [==============================] - 1s 19ms/step - loss: 0.1625 - accuracy: 0.9493 - val_loss: 0.2067 - val_accuracy: 0.9447
Epoch 3/50
55/55 [==============================] - 1s 24ms/step - loss: 0.1400 - accuracy: 0.9493 - val_loss: 0.2229 - val_accuracy: 0.9447
Epoch 4/50
55/55 [==============================] - 1s 17ms/step - loss: 0.1163 - accuracy: 0.9493 - val_loss: 0.2328 - val_accuracy: 0.9447
Epoch 5/50
55/55 [==============================] - 2s 34ms/step - loss: 0.0932 - accuracy: 0.9493 - val_loss: 0.2718 - val_accuracy: 0.9447
Epoch 6/50
55/55 [==============================] - 1s 19ms/step - loss: 0.0695 - accuracy: 0.9608 - val_loss: 0.2992 - val_accuracy: 0.9447
Epoch 7/50
55/55 [==============================] - 1s 13ms/step - loss: 0.0451 - accuracy: 0.9850 - val_loss: 0.3396 - val_accuracy: 0.9401
Epoch 8/50
55

In [17]:
best_hps.values

{'units1': 320,
 'dropout1': 0.0,
 'units2': 224,
 'dropout2': 0.4,
 'learning_rate': 0.001}